## Variational Thermofield Double State Generation

In [2]:
# Importing all of the necessary dependencies

import qiskit as 
import random
import numpy as np
import math
from scipy.optimize import minimize
from matplotlib import pyplot as plt
import scipy

The goal of this Notebook is to outline a the variational process outlined [this paper]('https://arxiv.org/abs/1906.02699') in order to generate Thermofield Double States (TFD states), which are defined as follows:

<br>
$$|TFD\rangle \ = \ \frac{1}{\sqrt{Z(\beta)}} \displaystyle\sum_{n} e^{- \beta E_n / 2} |E_n\rangle_A \otimes |E_n'\rangle_B$$
<br>

Notice how we are concered with the tensor product of two states for each term in the sum (we are concerned with two different systems of qubits, register $A$ and register $B$). We also define $H_A$ to be the Hamiltonian governing the dynamics of the qubits in the first register, with $H_A |E_n\rangle_A \ = \ E_n |E_n\rangle_A$, thus making our TFD state depend on both the energy eigenstates and the energy eigenvalues. It is also important to note that $\beta$ is defined as the inverse temperature of the system that we are describing, $\beta \ = \ 1/T$. $Z(\beta)$ denotes the partition function, which is given by:

<br>
$$Z(\beta) \ = \ \displaystyle\sum_{n} e^{- \beta E_n}$$
<br>

Finally, we define $E_n'\rangle_B$ to be a vector in the state space described by the second qubit register, $B$, specifically, the time-reversed counterpart of $|E_n\rangle_A$:

<br>
$$E_n'\rangle_B \ = \ \Theta |E_n\rangle_A \ = \ e^{i H_A t / \hbar} |E_n\rangle_A$$
<br>

**Part 2: Application**

We can begin by defining all of the necessary variables to run our simualtion, including the value of $\beta$, the transverse field strength, and the depth of the QAOA circuit. We will also use this block of code to intialize our two qubit registers, $A$ and $B$: